In [ ]:
#| hide
from backtest_sample.core import *

# backtest_sample

> backtest sample code: very simple with main features

This file will become your README and also the index of your documentation.

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install backtest_sample in Development mode

```sh
# make sure backtest_sample package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to backtest_sample
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/silvaac/backtest_sample.git
```

[repo]: https://github.com/silvaac/backtest_sample
[docs]: https://www.idatafactory.com/backtest_sample/

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs].

[repo]: https://github.com/silvaac/backtest_sample
[docs]: https://www.idatafactory.com/backtest_sample/

## How to use

Start by defining ...

In [ ]:
1+1

2